In [13]:
import pandas as pd
import numpy as np
import math

In [14]:
nba = pd.read_csv('NBA_Player_Stats_16-17.csv', encoding = "ISO-8859-1")
nba = pd.DataFrame(nba)

# Normalize quantitative variables
cols_to_norm = nba.columns[4:]
nba[cols_to_norm] = nba[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
nba

,Player,Team,Pos,Class,Age,GP,MPG,MIN_PERC,USG_PERC,Tor,...,3P_PERC,TS_PERC,PPG,RPG,TRB_PERC,APG,AST_PERC,SPG,BPG,VI
0,Aaron Brooks,Ind,PG,FrontCourt,0.619048,0.7875,0.265244,0.266471,0.362606,0.389908,...,0.562219,0.634543,0.158228,0.078014,0.170635,0.169643,0.361257,0.187192,0.053030,0.363636
1,Aaron Gordon,Orl,SF,BackCourt,0.095238,0.9750,0.722561,0.723280,0.388102,0.194954,...,0.431784,0.663329,0.401899,0.361702,0.380952,0.169643,0.183246,0.399015,0.189394,0.392045
2,Adreian Payne,Min,PF,BackCourt,0.285714,0.2000,0.076220,0.077599,0.470255,0.261468,...,0.299850,0.632040,0.110759,0.127660,0.563492,0.035714,0.139616,0.216749,0.147727,0.414773
3,AJ Hammons,Dal,C,BackCourt,0.238095,0.2500,0.073171,0.074671,0.317280,0.376147,...,0.749625,0.590738,0.069620,0.120567,0.523810,0.017857,0.066318,0.024631,0.223485,0.278409
4,Al Horford,Bos,C,BackCourt,0.523810,0.8250,0.832317,0.831625,0.376771,0.270642,...,0.532234,0.692115,0.443038,0.482270,0.468254,0.446429,0.425829,0.374384,0.477273,0.551136
5,Al Jefferson,Ind,C,BackCourt,0.619048,0.8000,0.277439,0.278184,0.558074,0.139908,...,0.000000,0.658323,0.256329,0.297872,0.666667,0.080357,0.191972,0.142857,0.090909,0.505682
6,Alan Anderson,Lac,SF,BackCourt,0.714286,0.3500,0.161585,0.161054,0.206799,0.169725,...,0.476762,0.618273,0.091772,0.056738,0.174603,0.035714,0.085515,0.049261,0.000000,0.215909
7,Alan Williams,Pho,PF,BackCourt,0.238095,0.5625,0.307927,0.307467,0.410765,0.240826,...,0.000000,0.684606,0.234177,0.439716,0.884921,0.044643,0.090750,0.280788,0.257576,0.426136
8,Alec Burks,Uta,SG,FrontCourt,0.285714,0.5000,0.323171,0.322108,0.461756,0.252294,...,0.493253,0.627034,0.212025,0.205674,0.420635,0.062500,0.130890,0.211823,0.045455,0.346591
9,Alex Abrines,Okc,SG,FrontCourt,0.190476,0.8250,0.320122,0.320644,0.269122,0.190367,...,0.571214,0.700876,0.189873,0.092199,0.178571,0.053571,0.095986,0.266010,0.049242,0.244318


In [15]:
train = nba.sample(frac=0.7)
train = pd.DataFrame(train)

test = nba.drop(train.index)
test = pd.DataFrame(test)

list_of_players = np.array(nba.iloc[0:]["Player"])
list_of_players_test = np.array(test.iloc[0:]["Player"])
list_of_players_train = np.array(train.iloc[0:]["Player"])
#random_state=nba.shape[0]

In [16]:
# Function returns tuple (actual distance from player 1, player 2 name)
def EuclidianDistance(Player_1, Player_2, data_set):
    player1 = data_set.loc[data_set['Player'] == Player_1]
    #player1.drop(player.columns[[0, 1, 2, 3]], axis=1, inplace= True)
                
    player2 = data_set.loc[data_set['Player'] == Player_2]
    #player2.drop(player.columns[[0, 1, 2, 3]], axis=1, inplace= True)

    distance = math.sqrt(sum(player1.iloc[0][4:24] - player2.iloc[0][4:24])**2)
    return distance, player2.iloc[0][0]

In [17]:
# itertate through all players and compare to Russell Westbrook. Store tuple of distances, player names 
def ClosestPlayersTo(Player_Name):
    distances = ()
    for i in range(len(list_of_players)):
        distances = distances + EuclidianDistance(Player_Name, list_of_players[i], nba)

    # convert stored tuples of distances and players into seperate numpy arrays and 
    # combine into pandas data frame and sort by dist.
    dist = []
    for i in range(0,len(distances), 2):
        dist.append(distances[i])
    dist = np.array(dist)
    players = []
    for i in range(1, len(distances), 2):
        players.append(distances[i])
    players = np.array(players)
    data = pd.DataFrame({'Distances':dist, 'Players':players})
    data = data.sort_values('Distances')
    return data.iloc[1:11]

In [18]:
###### Question 1
ClosestPlayersTo("Russell Westbrook")

,Distances,Players
175,0.604736,James Harden
270,1.887375,LeBron James
209,2.315093,John Wall
27,2.481884,Anthony Davis
145,2.600688,Giannis Antetokounmp
100,2.621476,DeMarcus Cousins
398,2.755040,Stephen Curry
236,3.015687,Karl-Anthony Towns
198,3.052168,Jimmy Butler
161,3.162755,Isaiah Thomas


In [19]:
#nba.loc[nba['Player'] == "LeBron James"]
#nba.loc[nba['Player'] == "John Wall"]

In [20]:
###### Question 2, KNN Algorithm

In [21]:
# Function returns actual distance from player 1, player 2
def EuclidianDistance_KNN(Player_1, Player_2):
    player1 = train.loc[train['Player'] == Player_1]
    #player1.drop(player.columns[[0, 1, 2, 3]], axis=1, inplace= True)
                
    player2 = test.loc[test['Player'] == Player_2]
    #player2.drop(player.columns[[0, 1, 2, 3]], axis=1, inplace= True)

    distance = math.sqrt(sum(player1.iloc[0][4:24] - player2.iloc[0][4:24])**2)
    return distance

In [23]:
# gets neigbors for only 1 observation in test set
import operator
def getNeighbors(train_set, Test_Player, k):
    distances = []
    #testInstance = list_of_players_test[0]
    for x in range(len(train_set)):
        dist = EuclidianDistance_KNN(str(train_set.iloc[x][0]), Test_Player)
        distances.append((train_set.iloc[x][0], dist))
    distances.sort(key=operator.itemgetter(1))
    #print(distances)
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors
    #print(neighbors)

In [24]:
# Get ALL Top K Neighbors for each observation in test set
def getALLNeighbors(train_set, k):
    neighbors_array = []
    #list_of_players_test = np.array(test.iloc[0:]["Player"])
    #list_of_players_train = np.array(train.iloc[0:]["Player"])
    
    for i in range(5): # Look at first 5 test observations only for now
        neighbors_array.append(getNeighbors(train, list_of_players_test[i], k))
    return neighbors_array

In [25]:
# Resize 1d list into mutlidimensional list
def to_matrix(l, n):
    return [l[i:i+n] for i in range(0, len(l), n)]
    #[myList[i:i+n] for i in range(0, len(myList), n)]

In [26]:
def GetLabels(neighbors_array, k):
    labels_train = []
    for i in range(len(neighbors_array)):
        for j in range(len(neighbors_array[i])):
            player = train.loc[train['Player'] == neighbors_array[i][j]]
            #print(player.iloc[0][2])
            labels_train.append(player.iloc[0][2])

    labels_train = to_matrix(labels_train, k)
    #print(labels_train)

    labels_test = []
    for i in list_of_players_test[0:5]: # Look at first 4 test observations only for now
        player = test.loc[test['Player'] == i] 
        label = player.iloc[0][2]
        labels_test.append(label)
    #print(labels_test)
    return labels_train, labels_test

In [27]:
# For now, use k = 9
neighbors_array = getALLNeighbors(train, 9)
knn_labels, true_labels = GetLabels(neighbors_array, 9)

In [28]:
#import operator
#def getResponse(neighbors):
    #classVotes = {}
    #for x in range(len(neighbors)):
        #response = neighbors[x][-1]
        #if response in classVotes:
            #classVotes[response] += 1
        #else:
            #classVotes[response] = 1
    #sortedVotes = sorted(classVotes.iteritems(), key=operator.itemgetter(1), reverse=True)
    #return sortedVotes[0][0]

In [29]:
#print(neighbors_array)
print(knn_labels)
print(true_labels)

[['PG', 'SF', 'SF', 'SG', 'SF', 'PF', 'C', 'SG', 'SF'], ['PF', 'C', 'SG', 'SF', 'C', 'PF', 'PG', 'PG', 'C'], ['SF', 'SG', 'SF', 'PF', 'C', 'PF', 'SF', 'SG', 'SG'], ['SF', 'SF', 'SF', 'SG', 'PF', 'C', 'PF', 'SG', 'SF'], ['C', 'SG', 'PF', 'C', 'SF', 'PF', 'SG', 'SF', 'SG']]
['C', 'PF', 'SG', 'SF', 'SF']


In [30]:
test[:5]

,Player,Team,Pos,Class,Age,GP,MPG,MIN_PERC,USG_PERC,Tor,...,3P_PERC,TS_PERC,PPG,RPG,TRB_PERC,APG,AST_PERC,SPG,BPG,VI
3,AJ Hammons,Dal,C,BackCourt,0.238095,0.2500,0.073171,0.074671,0.317280,0.376147,...,0.749625,0.590738,0.069620,0.120567,0.523810,0.017857,0.066318,0.024631,0.223485,0.278409
7,Alan Williams,Pho,PF,BackCourt,0.238095,0.5625,0.307927,0.307467,0.410765,0.240826,...,0.000000,0.684606,0.234177,0.439716,0.884921,0.044643,0.090750,0.280788,0.257576,0.426136
8,Alec Burks,Uta,SG,FrontCourt,0.285714,0.5000,0.323171,0.322108,0.461756,0.252294,...,0.493253,0.627034,0.212025,0.205674,0.420635,0.062500,0.130890,0.211823,0.045455,0.346591
11,Alex Poythress,Phi,SF,BackCourt,0.190476,0.0500,0.643293,0.644217,0.300283,0.112385,...,0.473763,0.685857,0.338608,0.333333,0.388889,0.071429,0.090750,0.246305,0.064394,0.301136
15,Alonzo Gee,Den,SF,BackCourt,0.476190,0.1375,0.057927,0.057101,0.113314,0.417431,...,0.000000,0.382979,0.025316,0.085106,0.369048,0.044643,0.143106,0.187192,0.030303,0.255682
